Results are per month or total integrated.

In [ ]:
import pandas as pd, numpy as np, matplotlib.dates as dates, seaborn as sns, paramiko, os, re, sys, tables, glob, sys, shutil
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from ipywidgets import interact, IntSlider
if "./src" not in sys.path:
    sys.path.insert(0, "./src")
from src import bcm_utils

# Define parameters

In [ ]:
#variables
data_path = "/Users/pkicsiny/sshfs/json_pickles"
year = 2018
rs = 10
months = [str(m).zfill(2) for m in list(range(4, 11))]
colnames = bcm_utils.get_column_names([24, 48], [rs])

#dirs
excel_brildata_path = "../excel_brildata/{}".format(year)
excel_json_path = "../excel_json/{}".format(year)
brilcalc_dir = "../brilcalc_data/{}_offline.csv".format(year)

#files in dirs, data from brildata hd5 files
excel_brildata_files = os.listdir(excel_brildata_path)
excel_brildata_charge_sums = [excel_brildata_files[idx]for idx, i in enumerate(["sums" in et for et in excel_brildata_files]) if i]
excel_brildata_charge_sum_ratios = [excel_brildata_files[idx]for idx, i in enumerate(["sum_ratio" in et for et in excel_brildata_files]) if i]
excel_brildata_sim_data_ratios = [excel_brildata_files[idx]for idx, i in enumerate(["sim_data" in et for et in excel_brildata_files]) if i]

In [ ]:
#files in dirs, data from webmonitor, only 2018, use this
excel_json_files = os.listdir(excel_json_path)
excel_json_charge_sums = [excel_json_files[idx]for idx, i in enumerate(["sums" in et for et in excel_json_files]) if i]
excel_json_charge_sum_errors = [excel_json_files[idx]for idx, i in enumerate(["sums_stat_uncertainties" in et for et in excel_json_files]) if i]
excel_json_charge_sum_ratios = [excel_json_files[idx]for idx, i in enumerate(["sum_ratio" in et for et in excel_json_files]) if i]
excel_json_sim_data_ratios = [excel_json_files[idx]for idx, i in enumerate(["sim_data_ratios_20" in et for et in excel_json_files]) if i]
excel_json_sim_data_stat_uncertainty_ratios = [excel_json_files[idx]for idx, i in enumerate(["sim_data_ratios_st" in et for et in excel_json_files]) if i]

## 1) Ratios -Z/+Z channels per month

In [ ]:
#calulate ratios for json charge data
for m in months:
    print(m)
    excel_one_month = pd.read_excel(os.path.join(excel_json_path, 'charge_sums_2018_{}.xlsx'.format(m)))
    excel_one_month["CH24/CH48"] = excel_one_month["CH24RS10"]/excel_one_month["CH48RS10"]
    ratios_one_month = excel_one_month[["Unnamed: 0", "CH24/CH48"]]
    ratios_one_month.columns = ["Fill", "CH24/CH48"]
    ratios_one_month.set_index("Fill", drop=True, inplace=True)
    #ratios_one_month.to_excel(excel_json_path+"/charge_sum_ratios_{}_{}.xlsx".format(year, m))

In [ ]:
#contains data from all excel files (per fill)
charge_sum_ratio_df = pd.DataFrame()

#contains data for each month
charge_sum_ratio_dict = {}

#contains data from all excel files (averaged per month)
charge_sum_ratio_monthly_df = pd.DataFrame()
for file in excel_json_charge_sum_ratios:
    print(file)
    excel_one_month = pd.read_excel(os.path.join(excel_json_path, file))
    
#add monthly dataframe to dict
    dict_key = file[-12:-5]
    charge_sum_ratio_dict[dict_key] = excel_one_month
    
#append to big dataframe
    charge_sum_ratio_df = charge_sum_ratio_df.append(excel_one_month)
    
#calculate mean and std of monthly data
    charge_sum_ratio_monthly_df = charge_sum_ratio_monthly_df.append(
        pd.concat([pd.DataFrame([round(excel_one_month.mean(), 3), round(excel_one_month.std(), 3)],
                                index=["mean", "σ"]).T], keys=[dict_key]))

In [ ]:
#mean and std for the whole year
pd.DataFrame([charge_sum_ratio_df.mean(), charge_sum_ratio_df.std()], index=["mean", "σ"]).T

In [ ]:
#mean and std per month
charge_sum_ratio_monthly_df.loc[(slice(None), ["CH24/CH48"]), :]

In [ ]:
#indexing for one month
pd.DataFrame(charge_sum_ratio_monthly_df.loc[(slice(None), ["CH24/CH48"]), :][:-2].mean(), columns=["CH24/CH48"]).round(3).T

## 2) Sim-data ratios per month

In [ ]:
#result df column names
ratios_dict, yearly_ratios_df, monthly_ratios_df, errors_dict, yearly_errors_df, monthly_errors_df = bcm_utils.sim_data_ratios_per_month(excel_json_sim_data_ratios,
                                                                                                  excel_json_sim_data_stat_uncertainty_ratios,
                                                                                              excel_json_path, colnames, 0, [0.5,1])

In [ ]:
monthly_ratios_df

In [ ]:
round(100*monthly_ratios_df["σ"]/monthly_ratios_df["mean"], 2)

In [ ]:
#mean and std for the whole year
pd.DataFrame([yearly_ratios_df.mean(), yearly_ratios_df.std()], index=["mean", "σ"]).T

In [ ]:
fig = bcm_utils.sim_data_ratio_plot(yearly_ratios_df, year, show_mean=False)
handle = fig.gca()
handle.set_ylim(0.5, 1.6)
fig

In [ ]:
#fig.savefig("../sim_data_ratio_plots/brildata_{}_offline.png".format(year))

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
plt.grid()
#plt.yticks(fontsize=13)
#plt.xticks(fontsize=13)

# assumes -Z +Z order in dataframe
labels = ["BLM -Z", "BLM +Z"]
ax.scatter(yearly_ratios_df.index, yearly_ratios_df.iloc[:, 0], c="b", label=labels[0])
ax.scatter(yearly_ratios_df.index, yearly_ratios_df.iloc[:, 1], c="r", label=labels[1])
ax.set_ylabel('Simulation/measurement', fontsize=16)
ax.set_xlabel('Proton-proton fills', fontsize=16)
ax.legend(fontsize=14)
ax.set_ylim(0.6, 1.5)
ax.text(7100, 0.71, r'$\bf{CMS}$ $\it{Preliminary}$', size=16)
#ax.get_xaxis().set_label_coords(0.86,-0.075)
#ax.get_yaxis().set_label_coords(-0.07,0.7)
ax.set_title(r'$\bf{{{CMS}}}$ Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=79.5 mb, $\sqrt{\mathrm{s}}$=13 TeV)', loc='right', fontsize=16)
#fig.savefig("../sim_data_ratio_plots/brildata_{}_offline_v2.png".format(year))

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
plt.grid()
#plt.yticks(fontsize=13)
#plt.xticks(fontsize=13)

# assumes -Z +Z order in dataframe
labels = ["BLM -Z", "BLM +Z"]
ax.scatter(yearly_ratios_df.index, yearly_ratios_df.iloc[:, 0], c="b", label=labels[0])
ax.scatter(yearly_ratios_df.index, yearly_ratios_df.iloc[:, 1], c="r", label=labels[1])
ax.set_ylabel('Simulation/measurement', fontsize=16)
ax.set_xlabel('Proton-proton fills', fontsize=16)
ax.legend(fontsize=14)
ax.set_ylim(0.6, 1.5)
#ax.text(7100, 0.71, r'$\bf{CMS}$ $\it{Preliminary}$', size=16)
#ax.get_xaxis().set_label_coords(0.86,-0.075)
#ax.get_yaxis().set_label_coords(-0.07,0.7)
ax.set_title(r'$\bf{CMS}$ Preliminary'+"            "+r'Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=79.5 mb, $\sqrt{\mathrm{s}}$=13 TeV)',
             loc='right', fontsize=13)
#fig.savefig("../sim_data_ratio_plots/brildata_{}_offline_v3.png".format(year))